## Setup

In [ ]:

import sys
import os
import logging

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# 2. Import your existing RAG services
from src.db import q_client, setup_collection, get_embedding
from src.services.ingestion import ingest_source
from src.services.retrieval import query_knowledge_base
from src.config import COLLECTION_NAME

# 3. Initialize Database
setup_collection()

# 4. Check Connection
try:
    info = q_client.get_collection(COLLECTION_NAME)
    print(f"SUCCESSFULLY CONNECTED!")
    print(f"Collection Name: {COLLECTION_NAME}")
    print(f"Current Chunk Count: {info.points_count}")
    print(f"Vector Size: {info.config.params.vectors.size}")
except Exception as e:
    print(f"Connection Failed: {e}")

 ## Ingest Sources

In [ ]:


sources = [
    
    # Example 1: The "Attention Is All You Need" paper
    "https://arxiv.org/pdf/1706.03762",
    
]

print("🚀 Starting Ingestion...\n")

for source in sources:
    print(f"🔹 Processing: {source}")
    try:
        result = ingest_source(source)
        print(f"{result}")
    except Exception as e:
        print(f"FAILED: {str(e)}")

count = q_client.count(collection_name=COLLECTION_NAME).count
print(f"\n Total Documents in DB: {count}")

## Ask Questions

In [ ]:

from IPython.display import display, Markdown

# --- ENTER YOUR QUESTION HERE ---
query = "What is the main topic of the paper?" 
# --------------------------------

print(f"🔎 Searching for: '{query}'...\n")

# Calls the function from src/services/retrieval.py
response = query_knowledge_base(query)

# Display formatted result
display(Markdown(f"### 🤖 Answer:\n{response}"))

## DANGER ZONE - Clear Database

In [ ]:

from qdrant_client.http import models

confirm = input("Type 'DELETE' to wipe the database: ")

if confirm == "DELETE":
    q_client.delete(
        collection_name=COLLECTION_NAME,
        points_selector=models.FilterSelector(
            filter=models.Filter() # Empty filter selects everything
        )
    )
    print("🗑️ Database wiped clean. Count is now 0.")
else:
    print("❌ Operation cancelled.")